In [46]:
import re
import math
import copy
from collections import deque

# Utils

In [47]:
def find_qubit(qubit, grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == qubit:
                return (i, j)
    return None

def ManDist(neighbor, qubit):
    return abs(neighbor[0] - qubit[0]) + abs(neighbor[1] - qubit[1]) - 1

def DoA(new_qubit, qubit_pos, grid, adj_mat):
    qubit = grid[qubit_pos[0]][qubit_pos[1]]
    return adj_mat[new_qubit-1][qubit-1] + adj_mat[qubit-1][new_qubit-1]

def update_neighbor_list(grid, X, Y, neighborList = set()):
    # Remove the occupied NN positions from neighborList.
    if (X, Y) in neighborList:
        neighborList.remove((X, Y))
    if Y > 0 and grid[X][Y-1] == 0:
        neighborList.add((X, Y-1))
    if X > 0 and grid[X-1][Y] == 0:
        neighborList.add((X-1, Y))
    if Y < len(grid[0])-1 and grid[X][Y+1] == 0:
        neighborList.add((X, Y+1))
    if X < len(grid)-1 and grid[X+1][Y] == 0:
        neighborList.add((X+1, Y))
    return neighborList

# Place Qubits

In [48]:
def place_qubits(pq, M, N, adj_mat):
    grid = [[0 for _ in range(N)] for _ in range(M)]
    placedQubitList = []

    # Place the first qubit in the center of the grid.
    grid[M//2][N//2] = pq.pop(0)
    placedQubitList.append((M//2, N//2))
    neighborList = update_neighbor_list(grid, M//2, N//2)
    # print(neighborList, placedQubitList)

    # Place the remaining qubits.
    while pq:
        q = pq.pop(0)
        minCost, minX, minY = int(1e9), int(1e9), int(1e9)
        for neighbor in neighborList:
            cost = 0
            for qubit in placedQubitList:
                cost += ManDist(neighbor, qubit) * DoA(q, qubit, grid, adj_mat)
            if cost < minCost:
                minCost = cost
                minX, minY = neighbor
            elif cost == minCost:
                if abs(neighbor[0] - M//2) + abs(neighbor[1] - N//2) < abs(minX - M//2) + abs(minY - N//2):
                    minX, minY = neighbor
            # print(f"qubit: {q}, neighbor: {neighbor}, cost: {cost}, minCost: {minCost}, minX: {minX}, minY: {minY}")
        grid[minX][minY] = q
        placedQubitList.append((minX, minY))
        neighborList = update_neighbor_list(grid, minX, minY, neighborList)
        # print(neighborList, placedQubitList)

    return grid


# Finding the best paths

In [49]:
# Check if cell(x,y) is in the grid
def is_valid_cell(grid, x, y):
    M, N = len(grid), len(grid[0])
    return 0 <= x < M and 0 <= y < N

def find_min_paths(grid, src, dest):
    M, N = len(grid), len(grid[0])
    directions = [(0, -1), (-1, 0), (0, 1), (1, 0)]
    queue = deque([(src, [src])])
    min_paths = []
    visited = set()

    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == dest:
            min_paths.append(path)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_cell(grid, nx, ny) and (nx, ny) not in visited:
                queue.append(((nx, ny), path + [(nx, ny)]))
        visited.add((x, y))
    return min_paths
    

# Rearrange Utils

In [50]:
def nPaths(p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    right_moves = abs(y1-y2)
    down_moves = abs(x1-x2)
    total_moves = right_moves + down_moves
    return int(math.factorial(total_moves) / (math.factorial(right_moves) * math.factorial(down_moves)))

def nTrace(p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    return abs(x1-x2) + abs(y1-y2) -1

def swap_qubits(grid, path):
    n = len(path) - 1
    grids = []
    for i in range(n):
        new_grid = copy.deepcopy(grid)
        swap_src, swap_dest = i, n-i-1
        count = 0

        for j in range(swap_src):
            x1, y1 = path[j][0], path[j][1]
            x2, y2 = path[j+1][0], path[j+1][1]
            new_grid[x1][y1], new_grid[x2][y2] = new_grid[x2][y2], new_grid[x1][y1]
            count += 1
        
        for j in range(swap_dest):
            x1, y1 = path[n-j][0], path[n-j][1]
            x2, y2 = path[n-j-1][0], path[n-j-1][1]
            new_grid[x1][y1], new_grid[x2][y2] = new_grid[x2][y2], new_grid[x1][y1]
            count += 1

        grids.append(new_grid)

    return grids

# Gate Level Interaction Routing

In [51]:
def rearrange_grid(grid, q1, q2, swaps):
    p1 = find_qubit(q1, grid)
    p2 = find_qubit(q2, grid)

    np = nPaths(p1, p2)
    nt = nTrace(p1, p2)
    pathList = find_min_paths(grid, p1, p2)
    # print(f"np: {np}, nt: {nt}, pathList: {pathList}")

    all_possible_grids = []
    for path in pathList:
        new_grids = swap_qubits(copy.deepcopy(grid), path)
        for new_grid in new_grids:
            all_possible_grids.append(new_grid)
    return (grid, all_possible_grids, swaps+nt)


# Main Function

In [52]:
filename = input("Enter the filename: ")
# filename = 'cnt3-5_180'
with open(f'benchmarks/revlib_decomposed/{filename}.real', 'r') as f:
    data = f.read()

qubits = int(re.search(r".numvars ([\d]*)", data).group(1))
gates = re.search(r"(?s)(?<=.begin).*?(?=.end)", data, re.MULTILINE).group(0)
lines = gates.strip().split('\n')

# print(qubits, lines)
adj_mat = [[0]*qubits for _ in range(qubits)]
counts = [0]*qubits
interactions = []

for t, line in enumerate(lines, 1):
    params = line.split(' ')
    if len(params) == 2: continue    
    qubit1 = int(re.search(r"\d+", params[1]).group(0))
    qubit2 = int(re.search(r"\d+", params[2]).group(0))
    interactions.append((qubit1, qubit2))

    adj_mat[qubit1-1][qubit2-1] += 1/t
    counts[qubit1-1] += 1

print('AdjMat:')
for row in adj_mat: print(row)
qubit_priority = []
interaction_factor = lambda x: sum(adj_mat[x]) + sum(row[x] for row in adj_mat)
possible_src = [i for i, count in enumerate(counts) if count == max(counts)]
if len(possible_src) == 1:
    qubit_priority.append(possible_src[0]+1)
else:
    qubit_priority.append(max(possible_src, key=interaction_factor)+1)

while len(qubit_priority) < qubits:
    last_qubit = qubit_priority[-1]
    max_val, max_qubit = 0, 0
    for i in range(qubits):
        if i+1 in qubit_priority: continue
        val = adj_mat[i][last_qubit-1] + adj_mat[last_qubit-1][i]
        if val > max_val:
            max_val, max_qubit = val, i+1
        elif val == max_val:
            if val == 0 or counts[i] > counts[max_qubit-1]:
                max_qubit = i+1
    qubit_priority.append(max_qubit)

print('Priority:', qubit_priority)

AdjMat:
[0, 0.16783216783216784, 0.6]
[0.39285714285714285, 0, 0.2658730158730158]
[0.2, 0.625, 0]
Priority: [2, 3, 1]


In [53]:
M, N = map(int, input("Enter the grid size: ").split())
# M, N = 3, 6
grid = place_qubits(qubit_priority, M, N, adj_mat)
print('Grid:')
for row in grid: print(row)

# rearrange_grid(grid, 1, 5, 2)
limit = int(input("Enter the limit: "))
# limit = 69
leaves = [([], grid, 0)] # (parent, grids, swaps)
for i, interaction in enumerate(interactions, 1):
    print(f"Line {i}: {interaction}")
    q1, q2 = interaction
    current, excluded = [], []
    for i in leaves:
        if ManDist(find_qubit(q1, i[1]), find_qubit(q2, i[1])) > 0:
            current.append(i)
        else:
            excluded.append(i)
    new_leaves = []

    for i in current:
        parent, grid, swaps = i
        new_parent, new_grids, new_swaps = rearrange_grid(grid, q1, q2, swaps)
        new_leaves.extend([(parent+[new_parent], new_grid, new_swaps) for new_grid in new_grids if new_swaps <= limit])

    leaves = excluded + new_leaves
    print(f"Leaves: {len(leaves)}")

Grid:
[1, 0]
[3, 2]
Line 1: (1, 3)
Leaves: 1
Line 2: (3, 2)
Leaves: 1
Line 3: (2, 1)
Leaves: 4
Line 4: (3, 1)
Leaves: 10
Line 5: (3, 2)
Leaves: 28
Line 6: (2, 1)
Leaves: 76
Line 7: (3, 2)
Leaves: 172
Line 8: (2, 3)
Leaves: 172
Line 9: (1, 3)
Leaves: 368
Line 10: (1, 2)
Leaves: 504
Line 11: (2, 3)
Leaves: 464
Line 12: (1, 2)
Leaves: 384
Line 13: (2, 3)
Leaves: 352


In [54]:
# Find the minimum swaps from the current grids
min_swaps = int(1e9)
for i in leaves:
    # print(i)
    parent, grids, swaps = i
    if swaps < min_swaps:
        min_swaps = swaps
        min_grid = grids
        min_parent = parent

print(f"Minimum swaps: {min_swaps}")
for row in min_grid: print(row)

print("Path: ")
for parent in min_parent:
    for row in parent: print(row)
    print()

Minimum swaps: 4
[1, 2]
[0, 3]
Parents: 
[1, 0]
[3, 2]

[3, 0]
[1, 2]

[0, 3]
[1, 2]

[1, 3]
[0, 2]

